In [1]:
#!/usr/bin/env python
# coding: utf-8
from time import gmtime, strftime

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from PIL import Image
import shutil
import os

In [2]:
# ---- My utils ----
import sys
sys.path.insert( 0, '../..' ) # To be able to access to utils
from models import *
from utils.train_arguments import *
from utils.utils_data import *
from utils.utils_training import *

usage: ipykernel_launcher.py [-h] [--verbose]
                             [--optimizer {adam,sgd,sgd_momentum,rmsprop}]
                             [--epochs EPOCHS] [--batch_size BATCH_SIZE]
                             [--learning_rate LEARNING_RATE]
                             [--min_learning_rate MIN_LEARNING_RATE]
                             [--get_path] [--weighted_loss]
                             [--plateau_scheduler] [--steps_best]
                             [--steps_scheduler] [--img_size IMG_SIZE]
                             [--crop_size CROP_SIZE] [--model_name MODEL_NAME]
                             [--pretrained]
                             [--validation_size VALIDATION_SIZE]
                             [--output_dir OUTPUT_DIR] [--data_augmentation]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/maparla/.local/share/jupyter/runtime/kernel-6c22a113-6eaa-43e1-b2bd-c133fa7be52e.json


Working with Jupyter notebook! (Default Arguments)


### Load data

In [3]:
args.img_size = 256
args.crop_size = 224
args.pretrained = True
args.batch_size = 128
args.validation_size = 0.15
args.get_path = True

In [4]:
if args.data_augmentation:
    train_aug = [
        transforms.ToPILImage(),  # because the input dtype is numpy.ndarray
        transforms.Resize((args.img_size, args.img_size)),
        transforms.RandomCrop((args.crop_size, args.crop_size)),
        transforms.RandomHorizontalFlip(0.5),  # because this method is used for PIL Image dtype
        transforms.RandomVerticalFlip(0.5),  # because this method is used for PIL Image dtype
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.ToTensor(),  # because inpus dtype is PIL Image
    ]
else:
    train_aug = [
        transforms.ToPILImage(),  # because the input dtype is numpy.ndarray
        transforms.Resize((args.img_size, args.img_size)),
        transforms.CenterCrop((args.crop_size, args.crop_size)),
        transforms.ToTensor(),  # because inpus dtype is PIL Image
    ]

val_aug = [
    transforms.ToPILImage(),  # because the input dtype is numpy.ndarray
    transforms.Resize((args.img_size, args.img_size)),
    transforms.CenterCrop((args.crop_size, args.crop_size)),
    transforms.ToTensor(),  # because inpus dtype is PIL Image
]

if args.pretrained:
    train_aug.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    val_aug.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))

# data_partition='', data_augmentation=None, validation_size=0.15, seed=42
train_dataset = SIMEPU_Dataset(data_partition='train', transform=transforms.Compose(train_aug), validation_size=args.validation_size)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, pin_memory=True, shuffle=True)

val_dataset = SIMEPU_Dataset(data_partition='validation', transform=transforms.Compose(val_aug), validation_size=args.validation_size)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, pin_memory=True, shuffle=False)

### Load model

In [5]:
args.model_name = "resnet34"
model_checkpoint = "../../results/resnet34_adam_256to224_lr0.0001_DA_pretrained_weightedLoss/model_best_accuracy.pt"

In [6]:
model = model_selector(args.model_name, num_classes=len(LABELS2TARGETS), pretrained=args.pretrained)
model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.load_state_dict(torch.load(model_checkpoint))
model = model.eval()

Pretrained-> Remember at end: transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm

#### Validation error

In [7]:
args.weighted_loss = True

In [8]:
if args.weighted_loss:
    print("Loaded Class weights!")
    with open("../../utils/class_weights_divide.pkl", "rb") as fp:  # Unpickling
        weights = pickle.load(fp)
    print("Weights: {}".format(weights))
    class_weights = torch.FloatTensor(weights).cuda()
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(weights, device='cuda'))
else:
    criterion = nn.CrossEntropyLoss()

Loaded Class weights!
Weights: [6.626878130217029, 6.4439935064935066, 3.1023837436498636, 43.14673913043478, 20.567357512953368, 16.006048387096776, 6.632414369256474, 21.284182305630026, 25.28343949044586]


In [9]:
current_val_loss, current_val_accuracy = val_step(val_loader, model, criterion)

In [10]:
print("-----------------------------")
print("Validation Accuracy {:.4f}%".format(current_val_accuracy))
print("-----------------------------")

-----------------------------
Validation Accuracy 93.6503%
-----------------------------


### Error Analysis

In [11]:
val_dataset = SIMEPU_Dataset(data_partition='validation', transform=transforms.Compose(val_aug),
                             validation_size=args.validation_size, get_path=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, pin_memory=True, shuffle=True)

In [12]:
model.eval()
val_loss, correct, total, count = 0, 0, 0, 0
errors = [] # Lista errores [["real","prediccion", "img_path"],...]
with torch.no_grad():
    for batch_idx, (inputs, targets, imgs_paths) in enumerate(val_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
                
        _, predicted = outputs.max(1)
        
        count += len(targets)
        np_targets = targets.data.cpu().numpy()
        np_predictions = predicted.data.cpu().numpy()
        different_index = np.argwhere(np_targets!=np_predictions)
        
        for indx in different_index:
            real = TARGETS2LABELS[np_targets[indx[0]]]
            predicted = TARGETS2LABELS[np_predictions[indx[0]]]
            path = imgs_paths[indx[0]]
            errors.append([real, predicted, path])

In [13]:
print("Total errors: {}".format(len(errors)))
print("{:.2f}% de error".format((len(errors)/count) * 100))

Total errors: 426
6.35% de error


In [14]:
if not os.path.exists("analysis"):
        os.makedirs("analysis")

In [15]:
for real, predicted, path in errors:
    if not os.path.exists("analysis/{}".format(real)):
        os.makedirs("analysis/{}".format(real))
    
    for i in range(2,999999999):
        destiny_name = "analysis/{}/{}_{}.jpg".format(real,predicted,i)
        if not os.path.exists(destiny_name):
            shutil.copy(path, destiny_name)
            break